In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from sklearn import metrics
import os
from matplotlib import patches


In [ ]:
# color_list = {0: '#1f77b4', 1: '#ff7f0e', 2: '#2ca02c', 3: '#d62728', 4: '#9467bd', 5: '#8c564b', 6: '#e377c2', 7: '#7f7f7f', 8: '#bcbd22', 9: '#17becf', -1: '#000000', -2: '#ff0000', -3: '#000000', -4: '#000000', -5: '#000000'}
# color_list = {0: 'b', 1: 'g', 2: 'r', 3: 'c', 4: 'm', 5: 'k', 6: 'k', 7: 'k', 8: 'k', }

from matplotlib import patches


plt.set_cmap('Set1')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
print(colors)
features_dir = '../features/'
datasets = ['cifar10', 'cifar100', 'dtd', 'svhn', 'tin', 'mnist', 'fashionmnist', 'notmnist', 'places365']
datasets_ = ['CIFAR-10', 'CIFAR-100', 'DTD', 'SVHN','Tiny ImageNet', 'MNIST', 'FashionMNIST', 'NOTMNIST', 'Places365']
# datasets = ['mnist', 'cifar100', 'dtd', 'svhn', 'tin', 'cifar-10', 'fashionmnist', 'notmnist', 'places365']
# datasets_ = ['MNIST', 'CIFAR-100', 'DTD', 'SVHN','Tiny ImageNet', 'CIFAR-10', 'FashionMNIST', 'NOTMNIST', 'Places365']
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
handles = []
ood = 5
for i, color in enumerate(colors):
    handles.append(patches.Patch(color=color, label=classes[i]))

handles.append(patches.Patch(color='#000000', label=datasets_[ood]))

In [ ]:
# Plot data with given mask.
# file = 'resnet18_cifar10_cifar100_dtd_svhn_tin_mnist_fashionmnist_notmnist_places365_tsne.npz'
# file = 'resnet18_mnist_cifar100_dtd_svhn_tin_cifar10_fashionmnist_notmnist_places365_tsne.npz'
file = 'lenet_cifar10_cifar100_dtd_svhn_tin_mnist_fashionmnist_notmnist_places365_tsne.npz'
# file = 'lenet_mnist_cifar100_dtd_svhn_tin_cifar10_fashionmnist_notmnist_places365_tsne.npz'
features = np.load(f'{features_dir}{file}')['features']
logits = np.load(f'{features_dir}{file}')['logits']
labels = np.load(f'{features_dir}{file}')['labels']
labels = labels.astype(int)

figure(figsize=(15,10), dpi=200)
mask = (labels >=0) # cifar10
plt.scatter(features[:,0][mask], features[:,1][mask], c=pd.Series(labels[mask]*1).map(dict(enumerate(colors))), s=10, alpha=0.9)
mask = (labels ==-ood) # cifar10
plt.scatter(features[:,0][mask], features[:,1][mask], c='k', s=10, alpha=0.9)
plt.legend(handles=handles)

In [ ]:

figure(figsize=(15,10), dpi=200)
mask = (labels >=0) # cifar10
plt.scatter(logits[:,0][mask], logits[:,1][mask], c=pd.Series(labels[mask]*1).map(dict(enumerate(colors))), s=10, alpha=0.9)
mask = (labels ==-ood) # cifar10
plt.scatter(logits[:,0][mask], logits[:,1][mask], c='k', s=10, alpha=0.9)
plt.legend(handles=handles)

In [ ]:
# files_ = ['cifar100', 'tin', 'places365', 'dtd', 'svhn', 'mnist','fashionmnist', 'notmnist']
# files = ['CIFAR-100', 'Tiny ImageNet', 'Places365', 'DTD', 'SVHN', 'MNIST',  'FashionMNIST', 'NOTMNIST']
files_ = ['fashionmnist', 'notmnist', 'cifar10', 'cifar100', 'tin', 'places365', 'dtd',  'svhn']
files = ['FashionMNIST', 'NOTMNIST', 'CIFAR-10', 'CIFAR-100', 'Tiny ImageNet', 'Places365', 'DTD', 'SVHN']
methods_ = ['msp', 'mls', 'react', 'knn', 'lof', 'mds', 'mds_2', 'odin', 'odin_2',]
methods = ['MSP', 'MLS', 'ReAct', 'KNN', 'LOF', 'MDS', 'MDS bez IP', 'ODIN', 'ODIN bez IP',]
plots = 9

data_list = []
for method in methods_:
    for file in files_:
        # data_list.append(np.load(f'../features/resnet18-{method}-cifar10/{file}.npz'))
        data_list.append(np.load(f'../features/resnet18-{method}-mnist/{file}.npz'))
        # data_list.append(np.load(f'../features/lenet-{method}-cifar10/{file}.npz'))
        # data_list.append(np.load(f'../features/lenet-{method}-mnist/{file}.npz'))

roc, pr_in, pr_out = [], [], []
for data in data_list:
    roc.append(metrics.RocCurveDisplay(fpr=data['fpr'], tpr=data['tpr']))

for data in data_list:
    pr_in.append(metrics.PrecisionRecallDisplay(precision=data['precision_in'], recall=data['recall_in']))

for data in data_list:
    pr_out.append(metrics.PrecisionRecallDisplay(precision=data['precision_out'], recall=data['recall_out']))

In [ ]:
fig, ax = plt.subplots(plots//3, plots//3, figsize=(15,10), dpi=100, sharey=True, sharex=True)
for i, disp in enumerate(roc):
    # disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'CIFAR-10 {files[i%len(files)]}')
    disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'MNIST {files[i%len(files)]}')
    
for i in range(len(methods_)):
    ax[i//3][i%3].set_title(f'Receiver operating characteristic - {methods[i]}')
    if i != 0:
        ax[i//3][i%3].get_legend().remove()
        
plt.show()

In [ ]:
fig, ax = plt.subplots(plots//3, plots//3, figsize=(15,10), dpi=100, sharey=True, sharex=True)
for i, disp in enumerate(pr_in):
    # disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'CIFAR-10 {files[i%len(files)]}')
    disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'MNIST {files[i%len(files)]}')
    
for i in range(len(methods_)):
    ax[i//3][i%3].set_title(f'''Precision-Recall curve - ID - {methods[i]}''')
    if i != 4:
        ax[i//3][i%3].get_legend().remove()
        
plt.show()

In [ ]:
fig, ax = plt.subplots(plots//3, plots//3, figsize=(15,10), dpi=100, sharey=True, sharex=True)
for i, disp in enumerate(pr_out):
    # disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'CIFAR-10 {files[i%len(files)]}')
    disp.plot(ax=ax[i//len(files_)//3][i//len(files_)%3], label=f'MNIST {files[i%len(files)]}')
    
for i in range(len(methods_)):
    ax[i//3][i%3].set_title(f'''Precision-Recall curve - OOD - {methods[i]}''')
    if i != 4:
        ax[i//3][i%3].get_legend().remove()
        
plt.show()

In [ ]:
import pandas as pd

methods = ['MSP', 'MLS', 'ReAct', 'KNN', 'LOF', 'MDS', 'MDS_2', 'ODIN', 'ODIN_2']
methods_ = ['msp', 'mls', 'react', 'knn', 'lof', 'mds', 'mds_2', 'odin', 'odin_2',]
ids = ['CIFAR-10', 'MNIST']
ids_ = ['cifar10', 'mnist']

dirs = []
for id in ids_:
    for method in methods_:
        dirs.append(f'{features_dir}lenet-{method}-{id}')

data = {}
for i, dir_ in enumerate(dirs):
    key = f'{ids_[i//len(methods_)]}_{methods_[i%len(methods_)]}'
    data[key] = {}
    for file in os.listdir(dir_):
        data[key][file[:-4]] = np.load(f'{dir_}/{file}')['distance']

df = pd.DataFrame()
for i, key in enumerate(data.keys()):
    datasets = data[key].keys()

    mean = {}
    mean['index'] = f'{ids[i//len(methods)]} {methods[i%len(methods)]}'
    for j, key2 in enumerate(datasets):
        mean[key2] = np.mean(data[key][key2])

    mean = pd.DataFrame([mean])
    df = pd.concat([df, mean])
    df.to_csv('../features/data.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import csv

import numpy as np
import pandas as pd
  
x, y = [], []
methods = ['MSP', 'MLS', 'ReAct', 'KNN', 'LOF', 'MDS', 'MDS bez IP', 'ODIN', 'ODIN bez IP']
methods_ = ['msp', 'mls', 'react', 'knn', 'lof', 'mds', 'mds_2', 'odin', 'odin_2']
ood_datasets_ = ['cifar10','cifar100','dtd','fashionmnist','mnist','notmnist','places365','svhn','tin']
ood_datasets = ['CIFAR-10', 'CIFAR-100', 'DTD', 'FashionMNIST', 'MNIST', 'NOTMNIST', 'Places365', 'SVHN', 'Tiny ImageNet']
ids = ['CIFAR-10', 'MNIST']
ids_ = ['cifar10', 'mnist']

with open('../features/data2.csv','r') as csvfile:
    next(csvfile)
    plots = csv.reader(csvfile, delimiter = ',')
      
    for row in plots:
        # x.append(row[0].split(' ')[1])
        y.append([float(z) for z in row[1:]])

x = ood_datasets
y, x = np.array(y), np.array(x)
plots = 9
fig, ax = plt.subplots(plots//3, plots//3, figsize=(20,10), dpi=100, sharey=True)
offset = 0
x_axis = np.arange(len(ood_datasets))
# xlim = [[0.6,1], [2,9], [2,7], [-0.5,0], [-4,-1], [-12000, 0], [-12000, 0], [0.1003,0.1009], [0.1003,0.1009], [-40,-10]]
xlim = [None, None, None, None, None, None, None, None, None, None]
for j in range(offset, plots + offset):
    mask = (np.arange(y.shape[0]) <9)
    # mask = (np.arange(y.shape[0]) % 11 == 8)
    x_ = [x for x, m in zip(x, mask) if m]
    x_ = np.array([x_[i:i+11] for i in range(0, len(x_), 11)])
    # y_.append([x for x, m in zip(y[:,j], mask) if m])
    mask = (np.arange(y.shape[0]) <9)
    y__ = [x for x, m in zip(y[j,:], mask) if m]
    # y__ = np.array([y__[i:i+11] for i in range(0, len(y__), 11)])
    # for k in range(9):
    #     # ax[j - offset].bar(x_axis+0.05*(k+1), y__[k], color = colors[k], width = 0.05, label = methods[k])
    #     print(y__[k], x_[0])
    # print(y__, x_[0])
    # df = pd.DataFrame(np.transpose(y__), columns=methods, index=x_[0])
    df = pd.DataFrame(np.transpose(y[j::9,:]), columns=ids, index=x_[0])
    # df = pd.DataFrame(y[::9,:], columns=ids, index=x_[0])
    if methods[j] not in ['MDS', 'MDS bez IP']:
        if j-offset == 4:
            df.plot.barh(figsize=(15,10), ax=ax[(j-offset)//3][(j-offset)%3], grid=True, 
                         xlim=xlim[j], title=f'Miara podobieństwa według metody {methods[j]}.').legend(loc='upper right', fontsize="14")
        else:
            df.plot.barh(figsize=(15,10), ax=ax[(j-offset)//3][(j-offset)%3], grid=True, legend=False, 
                         xlim=xlim[j], title=f'Miara podobieństwa według metody {methods[j]}.')
    else:
        # print(df)
        df['CIFAR-10'].plot.barh(figsize=(15,10), ax=ax[(j-offset)//3][(j-offset)%3], grid=True, legend=False, sharey=True, width=0.25, position=0.85,
                        xlim=xlim[-1], title=f'Miara podobieństwa według metody {methods[j]}.', color='#1f77b4')
        ax2 = ax[(j-offset)//3][(j-offset)%3].twiny()
        df['MNIST'].plot.barh(figsize=(15,10), ax=ax2, grid=True, legend=False, sharey=True, width=0.25, position=0.15,
                        xlim=xlim[j], title=f'Miara podobieństwa według metody {methods[j]}.', color='#ff7f0e')
        fig.tight_layout()
#     else:
#         if j-offset == 0:
#             df.plot.barh(figsize=(15,12), ax=ax[j-offset], xlabel=labels[j], sharey=True, grid=True, 
#                          xlim=[-40,10], title=f'''Zależność {labels[j]} od zbioru 
# OOD dla {"CIFAR-10" if mask[0] else "MNIST"} jako zbioru ID.''').legend(loc='center left')
#         else:
#             df.plot.barh(figsize=(15,12), ax=ax[j-offset], xlabel=labels[j], sharey=True, grid=True, legend=False, 
#                          xlim=[0,35] if j-offset == 1 else [50, 100], title=f'''Zależność {labels[j]} od zbioru 
# OOD dla {"CIFAR-10" if mask[0] else "MNIST"} jako zbioru ID.''')
#         plt.barh(x_axis+0.1*k, left=y__[k], color = colors[k], width = .1, height=0.1, label = methods[k])
    # ax[j - offset].tick_params(axis='x', labelrotation=90)
    # ax[j - offset].xticks(rotation = 90) 
    # ax[j - offset].set_xlabel('dataset')
    # plt.yticks(x_axis,x_[0])
    # if j == 0:
    #     ax[j - offset].set_ylim([25, 60]) 
    # else:
    #     ax[j - offset].set_ylim([80, 100])         
    # ax[j - offset].set_ylabel(labels[j])
    # plt.title(f'Zależność {labels[j]} od zbioru OOD dla zbioru {"CIFAR-10" if mask[0] else "MNIST"} jako ID.')
    # plt.legend()
